### Import dependencies

In [1]:
import pandas as pd
from path import Path

### Import Airline data

In [2]:
raw_2022_flights = pd.DataFrame()

# loop through csv files 
for x in range(1,8):
    file = Path(f'Resources/Flights_2022_{x}.csv')
    import_df = pd.read_csv(file,low_memory=False)
    raw_2022_flights = pd.concat([raw_2022_flights,import_df],ignore_index=True)

#### Export combined data into new csv

In [3]:
export_path = Path('Resources/Flights_2022_Master.csv')
raw_2022_flights.to_csv(export_path,index=False)

#### Re-Import master csv

In [4]:
messy_import_path = Path('Resources/Flights_2022_Master.csv')
messy_flight_df = pd.read_csv(messy_import_path,low_memory=False)